## Create global bioclimatic belts area

This notebook will use the 

In [ ]:
import pandas as pd
import re

In [ ]:
df = pd.read_csv("data/all_countries_areas_list.csv")

In [ ]:
import re

line = re.sub(r"([a-zA-Z]+)", r"'\1'", df.results.loc[0]).replace("=", ":")
area = eval(line)

In [ ]:
df = pd.DataFrame(area, columns=["ISOCode", "RSA", "PLAN"])
df = df.reset_index()

In [ ]:
df[df.index == 57]

In [ ]:
def normalize_dict_columns(df, column):
    """Extract column values"""

    # Extract groups from RSA column
    df1 = pd.json_normalize(df[column])

    # Extract group from groups and keep country index
    df2 = pd.json_normalize(df1["groups"]).reset_index()

    # Melt columns into rows in the group:[0,1,2,3], and keep groups index
    melted = pd.melt(df2, id_vars=["index"], value_vars=[0, 1, 2, 3]).reset_index()

    # Extract sum and group from value and keep the index
    normalized = pd.json_normalize(melted["value"]).reset_index()

    # Merge melted dataframe with "group" and "sum" df values
    result = pd.merge(melted, normalized, left_on="level_0", right_on="index")
    display(result.head(11))
    return result[["index_x", "variable", "group", "sum"]]

In [ ]:
plan_area_df = normalize_dict_columns(df, "PLAN")
rsa_df = normalize_dict_columns(df, "RSA")

In [ ]:
rsa_df.columns = ["contry_index", "variable", "group_rsa", "sum_rsa"]
plan_area_df.columns = ["contry_index", "variable", "group_plan", "sum_plan"]

In [ ]:
plan_area_df

In [ ]:
rsa_df

In [ ]:
# Take care, is merged with index
area_df = pd.merge(rsa_df, plan_area_df)
area_df

In [ ]:
final_result = pd.merge(df, area_df, left_on="index", right_on="contry_index")
final_result.to_csv("data/results.csv")